# Car Damage Assessment - Clean Working Version

This notebook contains only the essential, verified working components for car damage assessment with GPU support.

In [ ]:
# Complete GPU Setup & Verification for Car Damage Assessment
import os
import torch
import gc
from transformers import AutoProcessor
from unsloth import FastVisionModel
from datasets import load_dataset
from PIL import Image

print("🚀 === COMPLETE GPU SETUP & VERIFICATION ===")

# Import verification
print("✅ All required packages imported")

print("\n🧪 GPU Status Check:")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name()}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"✅ CUDA Version: {torch.version.cuda}")
else:
    print("❌ CUDA not available!")
    exit()

# Test basic GPU operations
print("\n🧪 Testing basic GPU operations...")
try:
    x = torch.randn(1000, 1000, device='cuda')
    y = torch.randn(1000, 1000, device='cuda')
    z = torch.mm(x, y)
    print("✅ Basic GPU operations working!")
    del x, y, z
    torch.cuda.empty_cache()
except Exception as e:
    print(f"❌ GPU operations failed: {e}")
    exit()

# Load and verify model
print("\n📦 Loading model and dataset...")
try:
    # Load the trained model
    model, tokenizer = FastVisionModel.from_pretrained(
        model_name="Kakyoin03/car-damage-assessment-llama-vision",
        load_in_4bit=True,
        device_map="auto",
    )
    FastVisionModel.for_inference(model)
    print("✅ Model loaded successfully!")
    
    # Load test dataset
    dataset = load_dataset("Kakyoin03/car_damage_detection_dataset", split="train")
    print(f"✅ Dataset loaded: {len(dataset)} samples")
    
    print(f"✅ Model type: {type(model)}")
    print(f"✅ Model device: {next(model.parameters()).device}")
    print(f"✅ Tokenizer loaded: {type(tokenizer)}")
    
except Exception as e:
    print(f"❌ Model/Dataset loading failed: {e}")
    print("🔄 This is expected if models aren't deployed yet")

print("\n🎉 EVERYTHING READY FOR GPU INFERENCE!")
print("✅ You can now run the inference code below.")
print("=" * 56)

In [ ]:
# Simple Working Inference Function
def analyze_car_damage_simple(image_path_or_pil, model, tokenizer):
    """
    Simple car damage analysis function that works reliably
    """
    try:
        # Clear GPU memory
        torch.cuda.empty_cache()
        
        # Load and prepare image
        if isinstance(image_path_or_pil, str):
            image = Image.open(image_path_or_pil).convert("RGB")
        else:
            image = image_path_or_pil.convert("RGB")
        
        # Resize if too large
        max_size = (1024, 1024)
        if image.size[0] > max_size[0] or image.size[1] > max_size[1]:
            image.thumbnail(max_size, Image.Resampling.LANCZOS)
        
        # Instruction
        instruction = """You are a car damage assessment expert. 
        Analyze this image and describe:
        1. What parts of the car are damaged
        2. Type of damage (scratch, dent, crack, etc.)
        3. Severity level (minor, moderate, major)
        Be concise and accurate."""
        
        # Prepare input
        messages = [
            {"role": "user", "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": instruction}
            ]}
        ]
        
        # Tokenize
        input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
        inputs = tokenizer(
            image,
            input_text,
            add_special_tokens=False,
            return_tensors="pt",
        ).to("cuda")
        
        # Generate
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=150,
                use_cache=False,
                temperature=0.8,
                top_p=0.9,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id if hasattr(tokenizer, 'pad_token_id') else tokenizer.eos_token_id
            )
        
        # Decode
        response = tokenizer.decode(output[0], skip_special_tokens=True)
        
        # Extract response
        if "assistant" in response:
            response = response.split("assistant")[-1].strip()
        
        # Clean up
        torch.cuda.empty_cache()
        
        return response
        
    except Exception as e:
        torch.cuda.empty_cache()
        return f"Analysis failed: {str(e)}"

print("✅ Simple inference function defined!")
print("📝 Usage: result = analyze_car_damage_simple(image, model, tokenizer)")

# Test if model is available
try:
    if 'model' in globals() and 'tokenizer' in globals():
        print("✅ Model and tokenizer are ready for testing!")
        
        # Test with first dataset image if available
        if 'dataset' in globals() and len(dataset) > 0:
            print("\n🧪 Testing with sample image...")
            test_image = dataset[0]["image"]
            result = analyze_car_damage_simple(test_image, model, tokenizer)
            print(f"📊 Sample result: {result[:100]}...")
    else:
        print("ℹ️ Model not loaded yet - run the setup cell first")
except Exception as e:
    print(f"ℹ️ Test skipped: {e}")

## Usage Examples

Once the model is loaded, you can use the inference function like this:

```python
# Analyze an image from file
result = analyze_car_damage_simple("path/to/image.jpg", model, tokenizer)
print(result)

# Analyze an image from dataset
test_image = dataset[0]["image"]
result = analyze_car_damage_simple(test_image, model, tokenizer)
print(result)
```